# Part 1

We reshape data by turning wide data to long data. We first drop an entirely empty column, then we rename the missing column names, and we continue on from there.

Next we answer related questions.

In [185]:
import pandas as pd
import numpy as np

bridges = pd.read_csv("https://raw.githubusercontent.com/MatthewFried/MAT5001/main/M11/M11_Data%20(1).csv")
bridges

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,NaN,delayed,62.0,12.0,20.0,102.0,305.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AMWEST,on time,694.0,4840.0,383.0,320.0,201.0
4,NaN,delayed,117.0,415.0,65.0,129.0,61.0


In [186]:
#drop row 2 since it was just a visual aid with no information
bridges.drop(2,inplace= True)

#rename column
bridges.rename(columns ={'Unnamed: 0' : 'Airline', 'Unnamed: 1' : 'Delay'}, inplace = True)

#fill in the Airline column
bridges['Airline'].fillna(method='ffill',inplace=True)

#reset the index
bridges_0 = bridges.reset_index(drop=True)

#check data
bridges_0

,Airline,Delay,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,ALASKA,delayed,62.0,12.0,20.0,102.0,305.0
2,AMWEST,on time,694.0,4840.0,383.0,320.0,201.0
3,AMWEST,delayed,117.0,415.0,65.0,129.0,61.0


In [187]:
#check the data types to see if conversion is necessary
#seems to be OK
bridges.dtypes

Airline           object
Delay             object
Los Angeles      float64
Phoenix          float64
San Diego        float64
San Francisco    float64
Seattle          float64
dtype: object

In [188]:
#test to see if there is a clearer path to tidy data by breaking up Delay column
bridges = pd.get_dummies(bridges_0, prefix=['Delay'], columns=['Delay'])
bridges

,Airline,Los Angeles,Phoenix,San Diego,San Francisco,Seattle,Delay_delayed,Delay_on time
0,ALASKA,497.0,221.0,212.0,503.0,1841.0,0,1
1,ALASKA,62.0,12.0,20.0,102.0,305.0,1,0
2,AMWEST,694.0,4840.0,383.0,320.0,201.0,0,1
3,AMWEST,117.0,415.0,65.0,129.0,61.0,1,0


In [189]:
#test to see if stacked data can assist in cleaning data
bridges.stack()

0  Airline          ALASKA
   Los Angeles         497
   Phoenix             221
   San Diego           212
   San Francisco       503
   Seattle            1841
   Delay_delayed         0
   Delay_on time         1
1  Airline          ALASKA
   Los Angeles          62
   Phoenix              12
   San Diego            20
   San Francisco       102
   Seattle             305
   Delay_delayed         1
   Delay_on time         0
2  Airline          AMWEST
   Los Angeles         694
   Phoenix            4840
   San Diego           383
   San Francisco       320
   Seattle             201
   Delay_delayed         0
   Delay_on time         1
3  Airline          AMWEST
   Los Angeles         117
   Phoenix             415
   San Diego            65
   San Francisco       129
   Seattle              61
   Delay_delayed         1
   Delay_on time         0
dtype: object

In [190]:
#test to see if there is a clearer path to tidy data by using melt
melted = pd.melt(bridges,['Airline'])
melted

,Airline,variable,value
0,ALASKA,Los Angeles,497.0
1,ALASKA,Los Angeles,62.0
2,AMWEST,Los Angeles,694.0
3,AMWEST,Los Angeles,117.0
4,ALASKA,Phoenix,221.0
5,ALASKA,Phoenix,12.0
6,AMWEST,Phoenix,4840.0
7,AMWEST,Phoenix,415.0
8,ALASKA,San Diego,212.0
9,ALASKA,San Diego,20.0


##Question 1.2

* For each city, which airline had the best on time performance?
* Which airline had the best overall on time performance?

In [191]:
#identify which columns hold the ontime performance, place it in a new dataframe
on_time = bridges[bridges['Delay_on time'] == 1]

#reset index
on_time = on_time.reset_index(drop=True)

#list of cities
cities = ['Los Angeles',	'Phoenix',	'San Diego',	'San Francisco',	'Seattle']

#identify which row is lower
on_time_performance = {}

for k in cities:
  if (on_time[k].iloc[0] - on_time[k].iloc[1] > 0):
    on_time_performance[k] = 'AMEWEST'
  else:
    on_time_performance[k] ='ALASKA'

print("Here we display the best on time performance for each city: ")
on_time_performance

Here we display the best on time performance for each city: 


{'Los Angeles': 'ALASKA',
 'Phoenix': 'ALASKA',
 'San Diego': 'ALASKA',
 'San Francisco': 'AMEWEST',
 'Seattle': 'AMEWEST'}

In [192]:
alaska = 0
amwest = 0

for k in on_time_performance.values():
  if (k == 'ALASKA'):
    alaska += 1
  else:
    amwest += 1

print(f"Here are the scores:\nALASKA\t%s\nAMWEST\t%s" %(alaska, amwest))

Here are the scores:
ALASKA	3
AMWEST	2


## Question 1.3

> What, if any, changes you would make to the visual presentation of the data if you were then asked to transform your “long” data back into a “wide”
format: would you mimic the structure of the graphic shown above? If not, how might you transform your “long” data to “wide” format to make its “wide” presentation easier to understand and work with? Provide an
example of your recommendation.

**Answer:** It really depends on what one is looking for.  The beauty of pivot tables in Excel is that exploring this question is just a matter of dragging and dropping. Under the circumstances, having data sourced from what looks like a McGraw Hill text, I will assume they were looking for exactly what they have. However, in the interest of not losing credit, I will make a possible transformation, based on potential other scrutiny and analysis one might explore.

In [193]:
#our current format
bridges_0

,Airline,Delay,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,ALASKA,delayed,62.0,12.0,20.0,102.0,305.0
2,AMWEST,on time,694.0,4840.0,383.0,320.0,201.0
3,AMWEST,delayed,117.0,415.0,65.0,129.0,61.0


In [194]:
#reformat so that the cities are rows and the Delay is next to each other all done by the lower portion displayed Airline
pivoted = bridges_0.pivot('Airline',cities,'Delay')
pivoted

Los Angeles,497.0,62.0,694.0,117.0
Phoenix,221.0,12.0,4840.0,415.0
San Diego,212.0,20.0,383.0,65.0
San Francisco,503.0,102.0,320.0,129.0
Seattle,1841.0,305.0,201.0,61.0
Airline,,,,
ALASKA,on time,delayed,NaN,NaN
AMWEST,NaN,NaN,on time,delayed


#Part 2

Solve accompanying problem.

In [195]:
#get data
df = pd.read_csv('https://raw.githubusercontent.com/MatthewFried/MAT5001/main/M11/Bridges.csv')
df.sample(5)

,Identifier,River,Location,Erected,Purpose,Length,Lanes,Clear,Through_or_Deck,Material,Span,Rel,Type
104,E91,O,44,1975,HIGHWAY,3756,6,G,THROUGH,STEEL,LONG,F,ARCH
50,E49,A,34,1902,HIGHWAY,1850,2,G,THROUGH,STEEL,MEDIUM,F,CANTILEV
41,E42,M,9,1895,HIGHWAY,2367,2,G,THROUGH,STEEL,LONG,F,SIMPLE-T
4,E6,M,23,1838,HIGHWAY,?,2,N,THROUGH,WOOD,?,S,WOOD
84,E76,M,6,1931,HIGHWAY,1500,4,G,THROUGH,STEEL,LONG,F,SUSPEN


###Task 2.1

For each river, your output should include the Purpose, Material, and count (aka ‘How Many?’), similar to the output shown in the graphic below for River 'A', and your report should include similar content for each of the rivers contained within the data set.

In [196]:
#not sure if giving it a name "How Many?" is necessary or not, so putting in as a print
#output in one line by using groupby with three keys and finding the count of any of the columns
print("\t\t\tHow many?\n", df['Identifier'].groupby([df['River'], df['Purpose'],df['Material']]).count())

			How many?
 River  Purpose   Material
A      AQUEDUCT  IRON         1
                 WOOD         3
       HIGHWAY   ?            1
                 IRON         2
                 STEEL       21
                 WOOD         8
       RR        IRON         1
                 STEEL        9
                 WOOD         2
       WALK      STEEL        1
M      HIGHWAY   IRON         4
                 STEEL       19
                 WOOD         3
       RR        IRON         2
                 STEEL       13
O      HIGHWAY   ?            1
                 IRON         1
                 STEEL        9
       RR        STEEL        4
Y      HIGHWAY   STEEL        2
       RR        STEEL        1
Name: Identifier, dtype: int64


###Task 2.2

Generate a second report that shows the average length for each ‘Purpose’/’Material’ bridge grouping within the data set. Recall that the ‘Length’ attribute is not provided to us in a numeric format and also contains
many missing values. Clean up the contents of that column and convert it to numeric format before attempting to generate your report

In [197]:
#we first confirm that the Length column is an object
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Identifier       108 non-null    object
 1   River            108 non-null    object
 2   Location         108 non-null    object
 3   Erected          108 non-null    int64 
 4   Purpose          108 non-null    object
 5   Length           108 non-null    object
 6   Lanes            108 non-null    object
 7   Clear            108 non-null    object
 8   Through_or_Deck  108 non-null    object
 9   Material         108 non-null    object
 10  Span             108 non-null    object
 11  Rel              108 non-null    object
 12  Type             108 non-null    object
dtypes: int64(1), object(12)
memory usage: 11.1+ KB


In [198]:
#we replace question marks with NaNs
df['Length'] = df['Length'].apply(lambda x: np.nan if x == '?' else x) 

#convert to numbers
df["Length"] = pd.to_numeric(df["Length"])

In [199]:
#we implement the same procedure as above, but only use two keys
print("\t\t\tAverage Length\n", df['Length'].groupby([df['Purpose'], df['Material']]).mean())

			Average Length
 Purpose   Material
AQUEDUCT  IRON        1000.000000
          WOOD        1092.000000
HIGHWAY   ?                   NaN
          IRON        1216.666667
          STEEL       1557.804348
          WOOD        1053.375000
RR        IRON        1100.000000
          STEEL       1946.850000
          WOOD                NaN
WALK      STEEL               NaN
Name: Length, dtype: float64


###Part 2.3

Break the data up into the suggested chart.

In [200]:
# define a function that specifies the aggregating functions we want to apply
# to the Length contained within each of our data1 quartile cuts which we make below
# Note the use of a dict object
def get_stats(group):
    return {'Average Length': group.mean(), 'Count': group.count(), 'Max Length': group.max(), 'Min Length': group.min()}

# group the Length column's values relative to the quartile cuts after making said quartile cuts
quartiles = pd.cut(df.Erected, 4)
grouped = df['Length'].groupby(pd.cut(df.Erected, 4))

# apply the get_stats function to the grouped values
# In the results we see the quartile cuts are used as the row index labels
grouped.apply(get_stats).unstack()

,Average Length,Count,Max Length,Min Length
Erected,,,,
"(1817.832, 1860.0]",1094.625000,8.0,1500.0,990.0
"(1860.0, 1902.0]",1603.347826,23.0,4558.0,1000.0
"(1902.0, 1944.0]",1676.181818,33.0,3000.0,860.0
"(1944.0, 1986.0]",1530.411765,17.0,3756.0,804.0


Commentary: Super impressive! But I can't say I ever would have figured this out with directly following class notes.